## Asymmetric Semantic Search

For asymmetric semantic search, you usually have a short query (like a question or some keywords) and you want to find a longer paragraph answering the query.

### Achtung

The language is important, some models focus on english, others need to be told which language to use. The one I used here is multilingual. It's slower but seems good.

In [18]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example.pdf"
loader = PyPDFLoader(file_path )
docs = loader.load()

print(len(docs))

17


In [74]:
from sentence_transformers import SentenceTransformer
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")


text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=embeddings
)

In [75]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 1},
)

In [76]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

model = ChatOllama(
  model="mistral-nemo"
)

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "Ho il profilo semplice. Quali sono le commissioni di trading su EXM?"})

results

{'input': 'Ho il profilo semplice. Quali sono le commissioni di trading su EXM?',
 'context': [Document(id='fdb9b549-b772-4d53-9545-bcf6d0458816', metadata={'source': './example.pdf', 'page': 15}, page_content='ALL1\n0524\n!\n!  \nINFORMAZIONI SUGLI ONERI E SUI COSTI  \n!\nDi seguito sono riportate le commissioni e le condizioni \neconomiche in vigore al  2 maggio 2024. Per successive \nmodifiche si rinvia al sito www.directa.it. \n \nCOMMISSIONI DI TRADING SUI DIVERSI MERCATI  \n \nEXM (ex MTA), EGM, MIV, ETFplus, GEM  \nProfili alternativi: \n•!Semplice: 5! per ordine eseguito \n•!Dinamica*: da 8 a 1,5!  \n•!Variabile: 1,9 per mille per ordine eseguito, con un  \nmassimo di 18! e un minimo di 1,5! (il minimo è di 5! per il \nmercato GEM) per ordini fino a 500.000!   \n \nATFund  \n•!Unico profilo disponibile : 1,9 per mille per ordine esegu ito, \ncon un massimo di 200! e un minimo di 5! per ordini fino a \n500.000!   \n  \nSEDEX e EuroTLX certificati \ndepositario Montetitoli \nProf

In [66]:
print(results["context"][0].metadata) # it's page + 1 actually

{'source': './example.pdf', 'page': 15}


In [78]:
rag_chain.invoke({"input": "Costi delle operazioni fuori mercato?"})

{'input': 'Costi delle operazioni fuori mercato?',
 'context': [Document(id='cfa69cda-b09f-463b-833d-0570e4da7d00', metadata={'source': './example.pdf', 'page': 16}, page_content="•! operazioni fuori mercato con controparte interna a directa       \n       50!  \n•! operazioni fuori mercato con controparte esterna a directa:  \n    - trade tra 0-!1 mln                          9 bps, minimo 50! \n    - trade tra !1 mln – !3 mln               7 bps, minimo 50! \n    - trade tra !3 mln – !5 mln               5 bps, minimo 50!               \n•!girata azionaria su titoli cartacei                                 80! \n \nSERVIZI DI ALERT VIA SMS, SMS TELEGRAM, EMAIL \n \nCon i servizi SMS all'eseguito, SMS a login, SMS Alert e e-mail \nAlert tieni il tuo conto sotto controllo. Puoi ricever e notifiche \nper ogni eseguito, per l’accesso al conto e, tramite gli Alert, \nsu soglie di prezzo da te impostate su titoli di tuo interesse.")],
 'answer': "Le operazioni fuori mercato con controparte